# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [9]:
from torch import nn, optim
import torch as t
import syft as sy

In [10]:
hook = sy.TorchHook(t)

In [5]:
# A Toy Dataset
data = t.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = t.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [6]:
# A Toy Model
model = nn.Linear(2,1)

In [7]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [8]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.4635)
tensor(1.4612)
tensor(0.9495)
tensor(0.6264)
tensor(0.4157)
tensor(0.2775)
tensor(0.1863)
tensor(0.1259)
tensor(0.0857)
tensor(0.0587)
tensor(0.0406)
tensor(0.0283)
tensor(0.0198)
tensor(0.0140)
tensor(0.0100)
tensor(0.0072)
tensor(0.0052)
tensor(0.0038)
tensor(0.0028)
tensor(0.0020)


# How to make this federated

> 1. Split our data into different pieces and send it to different workers

In [13]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [14]:
dataBob = data[0:2].send(bob)
targetBob = target[0:2].send(bob)

In [15]:
dataAlice = data[2:].send(alice)
targetAlice = data[2:].send(alice)

In [17]:
datasets = [(dataBob, targetBob), (dataAlice, targetAlice)]

In [21]:
datasets[0], datasets[1]

(((Wrapper)>[PointerTensor | me:14174892603 -> bob:18445251651],
  (Wrapper)>[PointerTensor | me:65993074018 -> bob:26123144840]),
 ((Wrapper)>[PointerTensor | me:3088829396 -> alice:51536374223],
  (Wrapper)>[PointerTensor | me:47895075561 -> alice:57108677266]))

In [32]:
model = nn.Linear(2,1)
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [33]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.1911,  0.4588]], requires_grad=True), Parameter containing:
 tensor([0.7054], requires_grad=True)]

In [23]:
_data, _target = datasets[0]

In [26]:
_data

(Wrapper)>[PointerTensor | me:14174892603 -> bob:18445251651]

In [24]:
_data.location

<VirtualWorker id:bob #objects:2>

> 2. Send model to data location

In [34]:
model = model.send(_data.location)

In [36]:
list(model.parameters())#notice the change

[Parameter containing:
 Parameter>[PointerTensor | me:16205244331 -> bob:97595607239],
 Parameter containing:
 Parameter>[PointerTensor | me:57844515130 -> bob:24379998427]]

In [37]:
opt.zero_grad()

In [38]:
pred = model(_data)

In [39]:
loss = ((pred - _target)**2).sum()

In [40]:
loss.backward()

(Wrapper)>[PointerTensor | me:94555604377 -> bob:93410575903]

In [41]:
opt.step()

> 3. Get model back

In [43]:
model = model.get()

In [44]:
loss.get()

tensor(0.0277, requires_grad=True)

# Let's package

In [45]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [46]:
train()

tensor(0.2696, requires_grad=True)
tensor(2.9203, requires_grad=True)
tensor(1.3898, requires_grad=True)
tensor(1.1896, requires_grad=True)
tensor(0.4829, requires_grad=True)
tensor(0.7953, requires_grad=True)
tensor(0.1853, requires_grad=True)
tensor(0.6528, requires_grad=True)
tensor(0.0863, requires_grad=True)
tensor(0.5978, requires_grad=True)
tensor(0.0551, requires_grad=True)
tensor(0.5739, requires_grad=True)
tensor(0.0470, requires_grad=True)
tensor(0.5615, requires_grad=True)
tensor(0.0467, requires_grad=True)
tensor(0.5540, requires_grad=True)
tensor(0.0486, requires_grad=True)
tensor(0.5489, requires_grad=True)
tensor(0.0511, requires_grad=True)
tensor(0.5451, requires_grad=True)
tensor(0.0534, requires_grad=True)
tensor(0.5422, requires_grad=True)
tensor(0.0554, requires_grad=True)
tensor(0.5400, requires_grad=True)
tensor(0.0570, requires_grad=True)
tensor(0.5382, requires_grad=True)
tensor(0.0584, requires_grad=True)
tensor(0.5369, requires_grad=True)
tensor(0.0595, requi